In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy
import sympy
from sympy import *
import pickle
import os
import re

import pandas as pd
from io import StringIO

from flowline_ode.sia_model import *
from flowline_ode.finite_differences import *

In [ ]:
def evaluate_rms_error(results, start_x=0):
    xs = results["xs"]
    zs = results["zs"]
    layer_solutions = results["layer_solutions"]
    layers_t0 = results["layers_t0_smoothed"]
    layers_t1 = results["layers_t1_smoothed"]

    xs = xs[xs >= start_x]

    layer_dl_dx, layer_dl_dt, _, _ = create_layer_finite_difference_fns(layers_t0, layers_t1)

    # Interpolate layer solutions to a grid defined by xs and zs
    interp_x = np.zeros((len(layer_solutions), len(xs)))
    interp_z = np.zeros_like(interp_x)
    interp_u = np.zeros_like(interp_x)
    interp_w = np.zeros_like(interp_x)
    for idx, layer_idx in enumerate(layer_solutions.keys()):
        interp_x[idx, :] = xs
        interp_z[idx, :] = layers_t0[layer_idx](xs)
        interp_u[idx, :] = layer_solutions[layer_idx].sol(xs) #layer_solution_velocity(layer_idx, xs)

        interp_w[idx, :] = layer_dl_dt(xs, layer_idx) + interp_u[idx,:]*layer_dl_dx(xs, layer_idx)

    # Gridded horizontal velocity from layer line solutions
    X, Z = np.meshgrid(xs, zs)
    u_mol_grid = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()),
                                            interp_u.flatten(), (X, Z), method='linear')
    w_mol_grid = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()),
                                            interp_w.flatten(), (X, Z), method='linear')

    # Gridded true horizontal and vertical velocity
    U_true = lambdify_and_vectorize_if_needed((results["x"], results["z"]), results["u"])(X, Z)
    W_true = lambdify_and_vectorize_if_needed((results["x"], results["z"]), results["w"])(X, Z)

    # Mask inside non-outer layers
    mask = np.nan * np.zeros_like(X)
    l1 = layers_t0[1](xs)
    l2 = layers_t0[-2](xs)
    mask[Z < np.maximum(l1, l2)] = 1
    mask[Z <= np.minimum(l1, l2)] = np.nan

    rms_err_horizontal = np.sqrt(np.nanmean((mask * (u_mol_grid - U_true*scipy.constants.year))**2))
    print(f"RMS error in horizontal velocity: {rms_err_horizontal:.2e} m/yr")

    rms_err_vertical = np.sqrt(np.nanmean((mask * (w_mol_grid - W_true*scipy.constants.year))**2))
    print(f"RMS error in vertical velocity: {rms_err_vertical:.2e} m/yr")

    return {"rms_err_horizontal": rms_err_horizontal, "rms_err_vertical": rms_err_vertical}

In [ ]:
starting_example = "outputs/20241023_125643_n2.0.pickle"
#starting_example = "outputs/20241023_125643_trials_with_gp/20241023_125643_n2.0.pickle"
#starting_example = "outputs/20240627_172129_n3.0.pickle"

In [ ]:
result_metrics = []

search_dir = os.path.dirname(starting_example)
base_filename = os.path.basename(starting_example)

for filename in os.listdir(search_dir):
    if filename.startswith(base_filename + ".rerun"):
        print(f"Found file: {filename}")

        # Try to extract SNR and slope from filename
        match = re.match(r"[\d\w.]+\.rerun_snr(\d+)_slope(\d+)_trial(\d+)\.pickle", filename)
        if match:
            snr = int(match.group(1))
            slope = int(match.group(2))
            trial_idx = int(match.group(3))
        else:
            print("Filename does not match expected format")
            continue

        print(f"SNR: {snr} dB, Slope: {slope} deg, Trial Index: {trial_idx}")

        with open(os.path.join(search_dir, filename), "rb") as f:
            results = pickle.load(f)

            metrics = evaluate_rms_error(results, start_x=50e3)
            metrics['snr'] = snr
            metrics['slope'] = slope
            metrics['trial_idx'] = trial_idx
            metrics['results'] = results
            result_metrics.append(metrics)


In [ ]:
df = pd.DataFrame(result_metrics)

initial_length = len(df)

df = df[df['rms_err_horizontal'] < 1e10]

print(f"Removed {initial_length - len(df)} entries with implausible RMS errors -- likely failed runs")

df

In [ ]:
df.pivot_table(index='snr', columns='slope', values='rms_err_horizontal', aggfunc='mean')

In [ ]:
import scipy.constants


def plot_velocity_error(results, fig, ax_horiz, ax_vert, show_cbar=True, vmax_horiz=5, vmax_vert=0.25, start_x=0):
    xs = results["xs"]
    zs = results["zs"]
    layer_solutions = results["layer_solutions"]
    layers_t0 = results["layers_t0_smoothed"]
    layers_t1 = results["layers_t1_smoothed"]

    xs = xs[xs >= start_x]

    layer_dl_dx, layer_dl_dt, _, _ = create_layer_finite_difference_fns(layers_t0, layers_t1)

    # Interpolate layer solutions to a grid defined by xs and zs
    interp_x = np.zeros((len(layer_solutions), len(xs)))
    interp_z = np.zeros_like(interp_x)
    interp_u = np.zeros_like(interp_x)
    interp_w = np.zeros_like(interp_x)
    for idx, layer_idx in enumerate(layer_solutions.keys()):
        interp_x[idx, :] = xs
        interp_z[idx, :] = layers_t0[layer_idx](xs)
        interp_u[idx, :] = layer_solutions[layer_idx].sol(xs) #layer_solution_velocity(layer_idx, xs)

        interp_w[idx, :] = layer_dl_dt(xs, layer_idx) + interp_u[idx,:]*layer_dl_dx(xs, layer_idx)

    # Gridded horizontal velocity from layer line solutions
    X, Z = np.meshgrid(xs, zs)
    u_mol_grid = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()),
                                            interp_u.flatten(), (X, Z), method='linear')
    w_mol_grid = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()),
                                            interp_w.flatten(), (X, Z), method='linear')

    # Gridded true horizontal and vertical velocity
    U_true = lambdify_and_vectorize_if_needed((results["x"], results["z"]), results["u"])(X, Z)
    W_true = lambdify_and_vectorize_if_needed((results["x"], results["z"]), results["w"])(X, Z)

    # Mask inside non-outer layers
    mask = np.nan * np.zeros_like(X)
    l1 = layers_t0[1](xs)
    l2 = layers_t0[-2](xs)
    mask[Z < np.maximum(l1, l2)] = 1
    mask[Z <= np.minimum(l1, l2)] = np.nan

    if ax_horiz:
        hv = mask * (u_mol_grid - U_true*scipy.constants.year)
        pcm_hz = ax_horiz.pcolormesh(X/1e3, Z, hv, cmap='coolwarm', vmin=-vmax_horiz, vmax=vmax_horiz)
        ax_horiz.set_xlabel("x [km]")
        ax_horiz.set_ylabel("z [m]")
        if show_cbar:
            cbar = fig.colorbar(pcm_hz, ax=ax_horiz)
            cbar.set_label("Horizontal velocity error\n[m/yr]")
    
    if ax_vert:
        vv = mask * (w_mol_grid - W_true*scipy.constants.year)
        pcm_vt = ax_vert.pcolormesh(X/1e3, Z, vv, cmap='coolwarm', vmax=vmax_vert, vmin=-vmax_vert)
        ax_vert.set_xlabel("x [km]")
        ax_vert.set_ylabel("z [m]")
        if show_cbar:
            cbar = fig.colorbar(pcm_vt, ax=ax_vert)
            cbar.set_label("Vertical velocity error\n[m/yr]")

    for ax in (ax_horiz, ax_vert):
        if ax:
            # Add layers
            for layer in results['layers_t0']:
                ax.plot(xs/1e3, layer(xs), linewidth=0.5, linestyle='--', color='k')
    

In [ ]:
low_snr_cases = (df[(df['snr'] == 1) & (df['slope'] == 15)])
low_snr_mean = low_snr_cases[['rms_err_horizontal', 'rms_err_vertical', 'snr']].mean()
low_snr_example = low_snr_cases.iloc[0]

high_snr_cases = (df[(df['snr'] == 20) & (df['slope'] == 0)])
high_snr_mean = high_snr_cases[['rms_err_horizontal', 'rms_err_vertical', 'snr']].mean()
high_snr_example = high_snr_cases.iloc[0]

fig = plt.figure(constrained_layout=True, facecolor='white', figsize=(12,5))
gs = fig.add_gridspec(2, 3, width_ratios=[0.3, 0.4, 0.3], wspace=0.05)

# Plot example from high SNR
ax_hv_high_snr = fig.add_subplot(gs[0, 0])
ax_vv_high_snr = fig.add_subplot(gs[1, 0])
plot_velocity_error(high_snr_example['results'], fig, ax_hv_high_snr, ax_vv_high_snr, start_x=50e3)
ax_hv_high_snr.set_xlim(50,100)
ax_vv_high_snr.set_xlim(50,100)

# Plot example from low SNR
ax_hv_low_snr = fig.add_subplot(gs[0, 2])
ax_vv_low_snr = fig.add_subplot(gs[1, 2])
plot_velocity_error(low_snr_example['results'], fig, ax_hv_low_snr, ax_vv_low_snr, start_x=50e3)
ax_hv_low_snr.set_xlim(50,100)
ax_vv_low_snr.set_xlim(50,100)

# Plot trends in RMS error
ax_hv_rms = fig.add_subplot(gs[0, 1])
df.pivot_table(index='snr', columns='slope', values='rms_err_horizontal', aggfunc='mean').plot(ax=ax_hv_rms, marker='o')
ax_hv_rms.set_title("Horizontal Velocity RMS Error")
ax_hv_rms.set_xlabel("SNR [dB]")
ax_hv_rms.set_ylabel("RMS Error [m/yr]")
ax_hv_rms.grid()
ax_hv_rms.invert_xaxis()
ax_hv_rms.legend(title="Slope [deg]")
ax_hv_rms.set_xlim(23, 0)

for example, label in ((low_snr_mean, '(e)'), (high_snr_mean, '(a)')):
    ax_hv_rms.scatter([example['snr']], [example['rms_err_horizontal']], s=150, facecolors='none', edgecolors='k', zorder=2)
    ax_hv_rms.annotate(label, np.array([example['snr']+2.5, example['rms_err_horizontal']-0.01]))

ax_vv_rms = fig.add_subplot(gs[1, 1])
df.pivot_table(index='snr', columns='slope', values='rms_err_vertical', aggfunc='mean').plot(ax=ax_vv_rms, marker='o')
ax_vv_rms.set_title("Vertical Velocity RMS Error")
ax_vv_rms.set_xlabel("SNR [dB]")
ax_vv_rms.set_ylabel("RMS Error [m/yr]")
ax_vv_rms.grid()
ax_vv_rms.invert_xaxis()
ax_vv_rms.legend(title="Slope [deg]")
ax_vv_rms.set_xlim(23, 0)

for example, label in ((low_snr_mean, '(f)'), (high_snr_mean, '(b)')):
    ax_vv_rms.scatter([example['snr']], [example['rms_err_vertical']], s=150, facecolors='none', edgecolors='k', zorder=2)
    ax_vv_rms.annotate(label, np.array([example['snr']+2.5, example['rms_err_vertical']-0.002]))


# Add layer line legend
import matplotlib.lines as mlines
layer_line = mlines.Line2D([], [], linewidth=0.5, linestyle='--', color='k', label='Layers')
ax_vv_high_snr.legend(handles=[layer_line], bbox_to_anchor=(0.3, -0.1))

# Add subplot labels
subfigure_labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']
for idx, ax in enumerate([ax_hv_high_snr, ax_vv_high_snr, ax_hv_rms, ax_vv_rms, ax_hv_low_snr, ax_vv_low_snr]):
    ax.set_title(subfigure_labels[idx], loc='left', fontweight='bold')

fig.savefig("figures/noise_examples.png", dpi=500, bbox_inches='tight')

In [ ]:
df.pivot_table(index='snr', columns='slope', values='rms_err_horizontal', aggfunc='mean').plot()

In [ ]:
df.pivot_table(index='snr', columns='slope', values='rms_err_vertical', aggfunc='mean').plot()

In [ ]:
# This is just temporary with values manually copied from Basal Slip Example to
# get an idea of what this plot will look like.

horizontal_velocity_csv_text = """
0 deg	5 deg	10 deg	15 deg
1 db	0.441	0.443	0.447	0.455
5 db	0.303	0.304	0.307	0.311
10 db	0.213	0.214	0.215	0.217
15 db	0.175	0.175	0.176	0.177
"""

vertical_velocity_csv_text = """
0 deg	5 deg	10 deg	15 deg
1 db	0.014	0.0157	0.0293	0.0583
5 db	0.00947	0.012	0.0276	0.0575
10 db	0.00645	0.00982	0.0268	0.0572
15 db	0.00503	0.00902	0.0266	0.0571
"""

# pandas read csv from text string

fig, (ax_horiz, ax_vert) = plt.subplots(2, 1, figsize=(5, 6))

df_horiz = pd.read_csv(StringIO(horizontal_velocity_csv_text), sep='\t')
df_horiz.plot(ax=ax_horiz)
ax_horiz.set_ylabel('RMS Errors in\nHorizontal Velocity [m/yr]')
ax_horiz.set_xlabel('SNR [dB]')
ax_horiz.set_title('RMS Errors in Horizontal Velocity vs SNR\nwith uncompensated cross-track layer slope')
ax_horiz.invert_xaxis()
ax_horiz.grid()

df_vert = pd.read_csv(StringIO(vertical_velocity_csv_text), sep='\t')
df_vert.plot(ax=ax_vert)
ax_vert.set_ylabel('RMS Errors in\nVertical Velocity [m/yr]')
ax_vert.set_xlabel('SNR [dB]')
ax_vert.set_title('RMS Errors in Vertical Velocity vs SNR\nwith uncompensated cross-track layer slope')
ax_vert.invert_xaxis()
ax_vert.legend(loc='upper left')
ax_vert.grid()

fig.tight_layout()
#fig.savefig('rms_errors_vs_snr.png', dpi=300)